# Parsing and Processing Lookup Responses

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import gzip
import json
from datetime import datetime

import multiprocess
import numpy as np
from tqdm import tqdm
import pandas as pd

from config import (
    inc_city_att, 
    inc_city_cl, 
    inc_city_verizon, 
    inc_city_el
)
from parsers import (
    cl_workflow, 
    att_workflow, 
    verizon_workflow, 
    el_workflow, 
    get_incorporated_places, 
    check_redlining, 
    get_holc_grade, 
    get_closest_fiber
)

In [3]:
# inputs
fn_acs = '../data/intermediary/census/aggregated_tables_plus_features.csv.gz'
pattern_att = '../data/intermediary/isp/att/*/*.geojson.gz' # pattern for all data collected from lookup tools
pattern_cl = '../data/intermediary/isp/centurylink/*/*.geojson.gz'
pattern_verizon = '../data/intermediary/isp/verizon/*/*.geojson.gz'
pattern_el = "../data/intermediary/isp/earthlink/*/*.geojson.gz"

# outputs
fn_att = "../data/output/speed_price_att.csv.gz"
fn_cl = '../data/output/speed_price_centurylink.csv.gz'
fn_verizon = '../data/output/speed_price_verizon.csv.gz'
fn_el = '../data/output/speed_price_earthlink.csv.gz'

# params
n_jobs = 20
recalculate = False

In [4]:
# This is from Census data we crunched in the previous notebook.
acs = pd.read_csv(fn_acs, dtype={'geoid': str, 'block_group': str})

# These are the columns we're going to bring to merge with lookup responses.
acs_cols = [
    'geoid', 'race_perc_non_white','income_lmi', 
    'ppl_per_sq_mile', 'n_providers', 'income_dollars_below_median',
    'internet_perc_broadband', 'median_household_income'
]

## Total data collected

In [5]:
def count_addresses(fn):
    """
    How many addresses did we successfully collect in each file?
    """
    import gzip
    import json
    count = 0
    with gzip.open(fn, 'rb') as f:
        for line in f.readlines():
            record = json.loads(line)
            if record['collection_status'] != 0:
                count += 1
    return count 

def count_successful_addresses(pattern, n_jobs=20):
    """
    For all files in `pattern`, sees how many addresses were successfully counted.
    Uses multiprocessing to speed things up.
    """
    files = glob.glob(pattern)
    count = 0
    with multiprocess.get_context("spawn").Pool(n_jobs) as pool:
        for _count in tqdm(pool.imap_unordered(count_addresses, files), 
                           total=len(files)):
            count += _count
    return count

In [6]:
att_count = count_successful_addresses(pattern_att, n_jobs=n_jobs)
verizon_count = count_successful_addresses(pattern_verizon, n_jobs=n_jobs)
cl_count = count_successful_addresses(pattern_cl, n_jobs=n_jobs)
el_count = count_successful_addresses(pattern_el, n_jobs=n_jobs)
all_records = att_count + verizon_count + cl_count + el_count 

print(f"""AT&T: {att_count}
Verizon: {verizon_count}
CenturyLink: {cl_count}
EarthLink: {el_count}
Total: {all_records}""")

100%|██████████| 17167/17167 [00:21<00:00, 783.85it/s]

AT&T: 456134
Verizon: 312357
CenturyLink: 245139
EarthLink: 593334
Total: 1606964


## Functions we're going to be using

We `check_redlining` grades by looking if an addresses' coordinates (converted to a Shapely `Point`) are within the `Polygon`s of redlining maps by Mapping Inequality. This actual check is done by `get_holc_grade`.

In [7]:
??get_holc_grade

Signature: get_holc_grade(row: dict, polygons: list) -> str
Source:   
def get_holc_grade(row: dict, 
                   polygons: list) -> str:
    """
    Converts any lat and lon in a dictionary into a shapely point,
    then iterate through a list of dictionaries containing 
    shapely polygons shapes for each HOLC-graded area.
    """
    point = Point(float(row['lon']), float(row['lat']))
    for polygon in polygons:
        if polygon['shape'].contains(point):
            return polygon['grade']
    return None
File:      ~/code/1_production-grade/isp/notebooks/parsers.py
Type:      function


## ATT

In [8]:
states = []

In [9]:
if not os.path.exists(fn_att) or recalculate:
    # find the data we collected for each block group.
    data_att = []
    files = glob.glob(pattern_att)
    with multiprocess.Pool(n_jobs) as pool:
        # create parallel jobs that parse each block group of data using `att_workflow`.
        for record in tqdm(pool.imap_unordered(att_workflow, files), 
                           total=len(files)):
            data_att.extend(record)
    att = pd.DataFrame(data_att)
    del data_att
    
    # only keep addresses in the incorporated city
    att = att[att.incorporated_place.isin(inc_city_att)]
    att['block_group'] = att['block_group'].apply(lambda x: f"{int(x):012d}")
    
    # check HOLC-grades for each address, and the distance to download speeds at or above 200 Mbps
    att = check_redlining(att)
    # merge census data, and save the file
    att_acs = att.merge(acs[acs_cols], how='left',
                        left_on='block_group', right_on='geoid')
    att_acs = att_acs[[c for c in att_acs.columns if c != 'geoid']]
    att_acs.to_csv(fn_att, index=False, compression='gzip')
else:
    att_acs = pd.read_csv(fn_att)

In [10]:
# start and end collection datetime
[datetime.fromtimestamp(att_acs.collection_datetime.min()), 
 datetime.fromtimestamp(att_acs.collection_datetime.max())]

[datetime.datetime(2022, 4, 18, 2, 18, 2),
 datetime.datetime(2022, 9, 27, 15, 27, 11)]

In [11]:
att_acs['state'].nunique()

20

In [12]:
len(att_acs)

432303

In [13]:
states.extend(att_acs['state'].unique())

In [14]:
att_acs.redlining_grade.value_counts(normalize=True)

C    0.504526
D    0.286795
B    0.159317
A    0.049362
Name: redlining_grade, dtype: float64

## Centurylink

In [15]:
if not os.path.exists(fn_cl) or recalculate:
    data_cl = []
    files = glob.glob(pattern_cl)
    with multiprocess.Pool(n_jobs) as pool:
        for record in tqdm(pool.imap_unordered(cl_workflow, files), 
                           total=len(files)):
            data_cl.extend(record)
    cl = pd.DataFrame(data_cl)
    del data_cl
    
    cl = cl[cl['incorporated_place'].isin(inc_city_cl)]
    cl = cl[cl.speed_down != 940]
    
    cl = check_redlining(cl)
    cl_acs = cl.merge(acs[acs_cols], how='left', 
                      left_on='block_group', right_on='geoid')
    cl_acs.to_csv(fn_cl, index=False, compression='gzip')
else:
    cl_acs = pd.read_csv(fn_cl)

<ipython-input-15-3a03e42623cc>:19: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  cl_acs = pd.read_csv(fn_cl)


In [16]:
# start and end collection datetime
[datetime.fromtimestamp(cl_acs.collection_datetime.min()), 
 datetime.fromtimestamp(cl_acs.collection_datetime.max())]

[datetime.datetime(2022, 4, 15, 18, 4, 41),
 datetime.datetime(2022, 4, 17, 17, 34, 25)]

In [17]:
len(cl_acs)

186611

In [18]:
cl_acs['state'].nunique()

15

In [19]:
states.extend(cl_acs['state'].unique())

In [20]:
cl_acs.redlining_grade.value_counts(normalize=True)

C    0.361080
B    0.358197
D    0.178315
A    0.102409
Name: redlining_grade, dtype: float64

## Verizon

In [21]:
if not os.path.exists(fn_verizon) or recalculate:
    data_verizon = []
    files = glob.glob(pattern_verizon)
    with multiprocess.Pool(n_jobs) as pool:
        for record in tqdm(pool.imap_unordered(verizon_workflow, files), 
                           total=len(files)):
            data_verizon.extend(record)
    verizon = pd.DataFrame(data_verizon)
    del data_verizon
    
    verizon = verizon[verizon.incorporated_place.isin(inc_city_verizon)]
    
    verizon['lon'] = verizon['lon'].astype(float)
    verizon['lat'] = verizon['lat'].astype(float)    
    verizon = check_redlining(verizon)
    
    verizon_acs = verizon.merge(acs[acs_cols], how='left',
                                left_on='block_group', right_on='geoid')
    verizon_acs.to_csv(fn_verizon, index=False, compression='gzip')
else:
    verizon_acs = pd.read_csv(fn_verizon)

<ipython-input-21-49db80972a30>:21: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  verizon_acs = pd.read_csv(fn_verizon)


In [22]:
# start and end collection datetime
[datetime.fromtimestamp(verizon_acs.collection_datetime.min()), 
 datetime.fromtimestamp(verizon_acs.collection_datetime.max())]

[datetime.datetime(2022, 4, 19, 17, 4, 55),
 datetime.datetime(2022, 4, 27, 23, 47, 27)]

In [23]:
verizon_acs['state'].nunique()

9

In [24]:
states.extend(verizon_acs['state'].unique())

In [25]:
verizon_acs.redlining_grade.value_counts(normalize=True)

C    0.418662
D    0.354534
B    0.202327
A    0.023830
E    0.000648
Name: redlining_grade, dtype: float64

## Earthlink

In [26]:
if not os.path.exists(fn_el) or recalculate:
    data_el = []
    files = glob.glob(pattern_el)
    with multiprocess.Pool(n_jobs) as pool:
        for record in tqdm(pool.imap_unordered(el_workflow, files), 
                           total=len(files)):
            data_el.extend(record)
    el = pd.DataFrame(data_el)
    del data_el
    
    el['block_group'] = el['block_group'].apply(lambda x: f"{int(x):012d}")
    el = check_redlining(el)    
    el_acs = el.merge(acs[acs_cols], how='left', 
                      left_on='block_group', right_on='geoid')
    el_acs = el_acs[el_acs.incorporated_place.isin(inc_city_el)]
    el_acs.to_csv(fn_el, index=False, compression='gzip')
else:
    el_acs = pd.read_csv(fn_el)

In [27]:
# start and end collection datetime
[datetime.fromtimestamp(el_acs.collection_datetime.min()), 
 datetime.fromtimestamp(el_acs.collection_datetime.max())]

[datetime.datetime(2022, 4, 20, 1, 45, 42),
 datetime.datetime(2022, 10, 1, 17, 0, 29)]

In [28]:
datetime.fromtimestamp(el_acs[~el_acs.major_city.isin(['nashville', 'columbus'])].collection_datetime.max())

datetime.datetime(2022, 5, 25, 17, 1, 44)

In [29]:
el_acs['state'].nunique()

35

In [30]:
# how many states did we collect data from?
states.extend(el_acs['state'].unique())
len(set(states))

46

In [31]:
# Which ISPs is EarthLink leasing from?
el_acs[el_acs.speed_down != 0].contract_provider.value_counts(dropna=False)

AT&T           285950
CenturyLink    112455
Frontier         8376
Name: contract_provider, dtype: int64

In [32]:
el_acs.incorporated_place.value_counts(normalize=True).head(20)

Chicago city                   0.103003
Houston city                   0.094457
Los Angeles city               0.080532
Phoenix city                   0.057785
Columbus city                  0.048118
Detroit city                   0.045808
Jacksonville city              0.041055
Charlotte city                 0.036138
Portland city                  0.034450
Seattle city                   0.032917
Oklahoma City city             0.032145
Kansas City city               0.030604
Las Vegas city                 0.030199
Denver city                    0.029746
New Orleans city               0.028129
Milwaukee city                 0.027319
Omaha city                     0.027114
Indianapolis city (balance)    0.026439
Albuquerque city               0.024725
Wichita city                   0.021730
Name: incorporated_place, dtype: float64

In [33]:
el_acs.technology.value_counts()

Fiber Based    169140
Fiber          137543
Copper         100098
Name: technology, dtype: int64

In [34]:
el_acs.redlining_grade.value_counts(normalize=True)

C    0.477026
D    0.263527
B    0.199072
A    0.060375
Name: redlining_grade, dtype: float64